# A functional burst search and download service

## Setup

In [1]:
%load_ext autoreload
%autoreload 2
import bursts
import asf_search as asf
from datetime import datetime, timedelta
import xarray as xr
from pathlib import Path
import pqdm
import numpy as np
from importlib import reload
from pqdm.threads import pqdm
import geopandas as gpd
from shapely import geometry, wkt
import time
import pystac
import fsspec
import xml.etree.ElementTree as ET
import time

In [2]:
def time_function_call(args,fun):
    start = time.time()
    result = fun(**args)
    end = time.time()
    time_s = end - start
    return result, time_s

In [3]:
safe_url = 'https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_SLC__1SDV_20220716T115019_20220716T115049_044125_054459_71A9.zip'

## Demoing interior objects
These objects will not likely not be accessed by users, but this function demonstrates how the metadata objects are created

In [ ]:
auth = bursts.get_netrc_auth()

In [ ]:
manifest, annotations = bursts.edl_download_metadata(safe_url, auth)

In [ ]:
slc = bursts.SLCMetadata(safe_url, manifest, annotations)

In [ ]:
swath = bursts.SwathMetadata(slc,'vv',swath_index=0)

In [ ]:
burst = bursts.BurstMetadata(swath,burst_index=0)

In [ ]:
item = burst.to_stac_item()
pystac.validation.validate(item)

## Creating burst metadata datasets
Users have the option of creating either a geodatframe, or a STAC catalog. Burst data download is only enabled for the STAC catalog, but the geodataframe is great for in-notebook visualizations. Hopefully, users won't have to create these datasets themselves and will instead use an ASF-provided dataset.

In [4]:
start = datetime.strptime('20210101','%Y%m%d')
end = datetime.strptime('20210301','%Y%m%d')
mt_edgecumb = 'POLYGON ((-135.7849 57.0417,-135.7243 57.0417,-135.7243 57.0654,-135.7849 57.0654,-135.7849 57.0417))'
insar_opts = dict(platform=[asf.PLATFORM.SENTINEL1], polarization=['VV','VV+VH','Dual VV'], beamMode='IW', processingLevel='SLC')
results = asf.geo_search(flightDirection='DESCENDING', intersectsWith=mt_edgecumb, start=start, end=end, **insar_opts)
safe_urls = [x.properties['url'] for x in results]

In [5]:
safe_urls

['https://datapool.asf.alaska.edu/SLC/SB/S1B_IW_SLC__1SDV_20210224T151554_20210224T151621_025750_0311EE_7026.zip',
 'https://datapool.asf.alaska.edu/SLC/SB/S1B_IW_SLC__1SDV_20210212T151554_20210212T151621_025575_030C38_5BC5.zip',
 'https://datapool.asf.alaska.edu/SLC/SB/S1B_IW_SLC__1SDV_20210131T151555_20210131T151621_025400_03067B_415D.zip',
 'https://datapool.asf.alaska.edu/SLC/SB/S1B_IW_SLC__1SDV_20210119T151555_20210119T151622_025225_0300ED_6AAB.zip',
 'https://datapool.asf.alaska.edu/SLC/SB/S1B_IW_SLC__1SDV_20210107T151555_20210107T151622_025050_02FB52_DB32.zip']

In [6]:
burst_list = bursts.get_burst_metadata(safe_urls, threads=5)

SUBMITTING | :   0%|          | 0/5 [00:00<?, ?it/s]

PROCESSING | :   0%|          | 0/5 [00:00<?, ?it/s]

COLLECTING | :   0%|          | 0/5 [00:00<?, ?it/s]

In [7]:
stac = bursts.generate_burst_stac_catalog(burst_list)
stac.describe()

* <Catalog id=burst-catalog>
    * <Collection id=372328_IW2>
      * <Item id=S1_SLC_20210224T151617_VV_372328_IW2>
      * <Item id=S1_SLC_20210212T151617_VV_372328_IW2>
      * <Item id=S1_SLC_20210131T151618_VV_372328_IW2>
      * <Item id=S1_SLC_20210119T151618_VV_372328_IW2>
      * <Item id=S1_SLC_20210107T151618_VV_372328_IW2>
    * <Collection id=372326_IW1>
      * <Item id=S1_SLC_20210224T151611_VV_372326_IW1>
      * <Item id=S1_SLC_20210212T151611_VV_372326_IW1>
      * <Item id=S1_SLC_20210131T151611_VV_372326_IW1>
      * <Item id=S1_SLC_20210119T151612_VV_372326_IW1>
      * <Item id=S1_SLC_20210107T151612_VV_372326_IW1>
    * <Collection id=372321_IW2>
      * <Item id=S1_SLC_20210224T151558_VV_372321_IW2>
      * <Item id=S1_SLC_20210212T151558_VV_372321_IW2>
      * <Item id=S1_SLC_20210131T151558_VV_372321_IW2>
      * <Item id=S1_SLC_20210119T151559_VV_372321_IW2>
      * <Item id=S1_SLC_20210107T151559_VV_372321_IW2>
    * <Collection id=372324_IW1>
      * <Item 

## Explore with Geopandas

In [8]:
items = [x.to_dict() for x in stac.get_all_items()]
df = gpd.GeoDataFrame.from_features(items, crs="epsg:4326")
df.index = [x['id'] for x in items]

In [9]:
in_aoi = df.intersects(wkt.loads(mt_edgecumb))
aoi = df.loc[in_aoi].copy()
print(f'Located {aoi.shape[0]} bursts that meet criteria')

Located 5 bursts that meet criteria


In [10]:
# aoi.explore(style_kwds={'fill':None})

## View using STAC-browser

In [11]:
stac_path = bursts.save_stac_catalog_locally(stac)

In [ ]:
bursts.initiate_stac_catalog_server(5555,stac_path)

## Download burst data
Using the STAC catalog, users can select an STAC item, then pass this item to `bursts.edl_download_burst_data`. The data is returned in a 2d numpy array with a complex float data type.

In [12]:
item_list = [stac.get_item(x,recursive=True) for x in aoi.index]

In [16]:
stack_dataset = bursts.edl_download_stack(item_list, threads=5)

SUBMITTING | :   0%|          | 0/5 [00:00<?, ?it/s]

PROCESSING | :   0%|          | 0/5 [00:00<?, ?it/s]

COLLECTING | :   0%|          | 0/5 [00:00<?, ?it/s]

In [17]:
stack_dataset

<xarray.Dataset>
Dimensions:                               (line: 1519, sample: 24229, time: 5)
Coordinates:
  * line                                  (line) int64 0 1 2 ... 1516 1517 1518
  * sample                                (sample) int64 0 1 2 ... 24227 24228
  * time                                  (time) datetime64[ns] 2021-02-24T15...
Data variables:
    S1_SLC_20210224T151610_VV_372326_IW3  (line, sample) complex128 0j 0j ... 0j
    S1_SLC_20210212T151610_VV_372326_IW3  (line, sample) complex128 0j 0j ... 0j
    S1_SLC_20210131T151610_VV_372326_IW3  (line, sample) complex128 0j 0j ... 0j
    S1_SLC_20210119T151611_VV_372326_IW3  (line, sample) complex128 0j 0j ... 0j
    S1_SLC_20210107T151611_VV_372326_IW3  (line, sample) complex128 0j 0j ... 0j

In [ ]:
# stack_dataset.to_zarr('example.zarr')

## xarray scrap

In [ ]:
np.save('./example_burst_data.npy',arrays[0])

In [ ]:
item = items[0]
array = np.load('./example_burst_data.npy')

In [ ]:
n_lines, n_samples = array.shape
dims = ('time','line','sample')
coords = ([item.datetime],list(range(n_lines)),list(range(n_samples)))
coords = {key:value for key,value in zip(dims,coords)}
burst_array = xr.DataArray(np.expand_dims(array,axis=0),coords=coords,dims=('time','line','sample'),attrs=item.properties)

In [ ]:
burst_array